[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JorisRoels/deep-learning-biology/blob/main/exercises/assignments/2020-dlb-5-unsupervised-neural-networks.ipynb)

# Exercise 5: Unsupervised neural networks

In this notebook, we will be see how autoencoders can be used for clustering and semi-supervised classification. 

The structure of these exercises is as follows: 

1. [Import libraries and download data](#scrollTo=ScagUEMTMjlK)
2. [Data pre-processing](#scrollTo=ohZHyOTnI35b)
3. [Building an autoencoder](#scrollTo=kIry8iFZI35y)
4. [Clustering with autoencoders](#scrollTo=uXrEb0rTI35-)
5. [Semi-supervised classification](#scrollTo=kEU34VvDUPr_)

## 1. Import libraries and download data
Let's start with importing the necessary libraries. 

In [ ]:
import pickle
import numpy as np
import random
import os
import matplotlib.pyplot as plt
plt.rcdefaults()
import pandas as pd
from sklearn.metrics import adjusted_rand_score
from sklearn.model_selection import train_test_split
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm, metrics
from progressbar import ProgressBar, Percentage, Bar, ETA, FileTransferSpeed
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms
import gdown
import zipfile
import os

As you will notice, Colab environments come with quite a large library pre-installed. If you need to import a module that is not yet specified, you can add it in the previous cell (make sure to run it again). If the module is not installed, you can install it with `pip`. 

To make your work reproducible, it is advised to initialize all modules with stochastic functionality with a fixed seed. Re-running this script should give the same results as long as the seed is fixed. 

In [ ]:
# make sure the results are reproducible
seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# run all computations on the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Running computations with %s' % torch.device(device))
if torch.cuda.is_available():
    print(torch.cuda.get_device_properties(device))

## 2. Data pre-processing

We will use the well-known MNIST benchmarking dataset. This dataset is well-known in the world of machine learning. It consists of relatively small $28 \times 28$ pixel images of digits. The goals of the dataset is to classify the digit images to the actual number that is on the image. 

In [ ]:
# batch size for the dataloader
batch_size = 64

# data loaders
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True, 
                   transform=transforms.ToTensor()), batch_size=batch_size, 
                   shuffle=False)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, download=True, 
                   transform=transforms.ToTensor()), batch_size=batch_size, 
                   shuffle=False)

# print statistics
print('The train data consists of %d samples' % len(train_loader.dataset))
print('The test data consists of %d samples' % len(test_loader.dataset))

Here are a few examples of the data, the corresponding label is on top of each image. 

In [ ]:
# visualize the data
n = 5
for i in range(n):
    plt.subplot(1, n, i+1)
    plt.imshow(test_loader.dataset[i][0].numpy()[0, ...], cmap='gray')
    plt.title(test_loader.dataset[i][1])
    plt.axis('off')
plt.show()

## 3. Building an autoencoder

Now, we have to implement the autoencoder and train it. Let's start by defining the architecture. 

**Exercise**: build an autoencoder with a single hidden layer. Not that every pixel in the image corresponds to an input dimension, i.e. the input and output of the autoencoder will be $28 \times 28$ dimensional and images will have to be reshaped in vector format. Implement the following architecture: 
- The first layer (encoder) will be a [fully connected layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) with [sigmoid](https://pytorch.org/docs/stable/generated/torch.nn.Sigmoid.html) activation that transforms the input features to a 64 dimensional (hidden) feature vector representation. 
- The second layer (decoder) is another [fully connected layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) that transforms the hidden representation to the original input dimensions. 
- Print the network architecture and propagate a test sample to validate your architecture. 

In [ ]:
class AutoEncoder(nn.Module):
    
    def __init__(self, dim_input, dim_hidden):
        super().__init__()

        """
        INSERT CODE HERE
        """

    def forward(self, x):

        """
        INSERT CODE HERE
        """

        return x

dim_hidden = 64
"""
INSERT CODE HERE
"""

# propagate a sample
"""
INSERT CODE HERE
"""

# show the result
"""
INSERT CODE HERE
"""

Time to train the network now! Remember, a single training iteration consists of the following steps: 
1. Construct a batch
2. Zero the gradients
3. Forward propagation
4. Compute the loss
5. Backward propagation
6. Update the weights

**Exercise**: train the autoencoder for 10 epochs on the data. 
- Map the autoencoder to the computing device
- Define an Adam optimizer, with a learning rate of $10^{-2}$ and weight decay of $10^{-5}$
- Define a mean squared error (MSE) loss function
- Fill in the missing code in the `train_epoch` and `test_epoch` functions
- Run the autoencoder on a sample and show the obtained reconstruction

In [ ]:
# train for one epoch
def train_epoch(net, loader, optimizer, criterion, device): 
    net.train()
    loss_avg = 0
    for x, _ in loader:
        """
        INSERT CODE HERE
        """

    loss_avg = loss_avg / len(loader)
    return loss_avg

# test for one epoch
def test_epoch(net, loader, criterion, device): 
    net.eval()
    loss_avg = 0
    for x, _ in loader:
        """
        INSERT CODE HERE
        """

    loss_avg = loss_avg / len(loader)
    return loss_avg

# train the autoencoder
def train(net, train_loader, test_loader, optimizer, epochs, criterion, device): 
    train_losses = np.zeros((epochs))
    test_losses = np.zeros((epochs))
    for epoch in range(epochs):
        train_losses[epoch] = train_epoch(net, train_loader, optimizer, criterion, device)
        test_losses[epoch] = test_epoch(net, test_loader, criterion, device)
        print("Epoch %d/%d: Train loss = %.4f - Test loss = %.4f" 
              % (epoch + 1, epochs, train_losses[epoch], test_losses[epoch]))
    
    return train_losses, test_losses

# parameters
lr = 1e-2
weight_decay = 1e-5
epochs = 10

# computing device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# map the network to the device
"""
INSERT CODE HERE
"""

# optimizer
"""
INSERT CODE HERE
"""

# loss function
"""
INSERT CODE HERE
"""

# start training
train_loss, test_loss = train(net_ae, train_loader, test_loader, 
                              optimizer, epochs, criterion, device)

# visualize training result
plt.plot(train_loss)
plt.plot(test_loss)
plt.legend(('Train', 'Test'))
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.show()

In [ ]:
# propagate a sample
"""
INSERT CODE HERE
"""

# show the result
"""
INSERT CODE HERE
"""

## 4. Clustering with autoencoders

In this section, we will show you how you can cluster data with an autoencoder model. Perhaps the simplest approach is to cluster the data in the encoded space, e.g. using k-means clustering. 

**Exercise**: Cluster the MNIST test data using the encodings and k-means clustering
- Compute the embeddings of the test data and store them in a numpy array
- Apply k-means clustering on the embeddings
- Evaluate the clustering with the [adjusted rand score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html#sklearn.metrics.adjusted_rand_score). 

In [ ]:
def compute_embeddings(net, loader):
    """
    INSERT CODE HERE
    """
    return embeddings

# compute the embeddings of the autoencoder on the test set
"""
INSERT CODE HERE
"""

# cluster the embeddings
"""
INSERT CODE HERE
"""

# evaluate
"""
INSERT CODE HERE
"""

The following code will compare the true labels and the clustering results. Note that the colors do not necessarily correspond to the same cluster. 

In [ ]:
# reduce the dimensionality of the hidden representations
h_red = TSNE(n_components=2, random_state=seed).fit_transform(h[:1000])

# visualize the reduced representations and label each sample
plt.subplot(1, 2, 1)
scatter = plt.scatter(h_red[:, 0], h_red[:, 1], c=labels[:1000], cmap='Paired')
plt.legend(handles=scatter.legend_elements()[0], labels=range(10))
plt.title('True labels')
plt.subplot(1, 2, 2)
scatter = plt.scatter(h_red[:, 0], h_red[:, 1], c=predictions[:1000], cmap='Paired')
plt.title('Clustering')
plt.show()

We will now focus towards the extracted features of the autoencoder. Within the encoder, there are connections to each pixel for each hidden variable. In other words, we can visualize which pixels are important to activate a particular hidden variable. 

**Exercise**: Visualize the extracted features based on the matrix weights from the encoder. 
- Extract the encoder matrix parameter values
- For each hidden variable, reshape the corresponding features connected to the input to a $28 \times 28$ image 
- Visualize all these images

In [ ]:
# extract the matrix parameters
"""
INSERT CODE HERE
"""

# visualize the features
"""
INSERT CODE HERE
"""

## 5. Semi-supervised classification

Semi-supervised classification aims to use large amounts of unlabeled data and a limited amount of labeled data for maximum classification performance. We will see that the extracted features of an autoencoder can serve as suitable features for classification. 

**Exercise**: Train and validate an SVM for increasing amounts of labeled training data. 
- Compute the embeddings of the train and test data. 
- Implement the `train_svm` function that trains an SVM with training $n$ samples. 
- Implement the `test_svm` function that tests an SVM on the complete test set and reports the accuracy. 
- Train an SVM for various amounts of labeled data with the autoencoder embeddings as input. 
- As a baseline, compare this approach to an SVM that is trained on the actual pixel values instead of the encodings. 

In [ ]:
def train_svm(x_train, y_train, n):
    """
    INSERT CODE HERE
    """
    return clf

def test_svm(clf, x_test, y_test):
    """
    INSERT CODE HERE
    """
    return acc

# compute the embeddings of the autoencoder on the train and test set
"""
INSERT CODE HERE
"""

# train and test an SVM for a varying amount of training data
n_train_min = 10
n_train_max = 200
n_train_step = 10
n_samples = np.arange(n_train_min, n_train_max, n_train_step)
accuracies_ae = np.zeros((len(n_samples)))
accuracies_bl = np.zeros((len(n_samples)))
for j in range(len(n_samples)):
    # autoencoder
    """
    INSERT CODE HERE
    """
    # baseline
    """
    INSERT CODE HERE
    """

# visualize the results
"""
INSERT CODE HERE
"""